# Factorisation de matrice et recommandations

Le notebook utilise la factorisation de matrice pour calculer des recommandations sur la base [movielens](https://grouplens.org/datasets/movielens/). On utilise le jeu de données [ml-latest-small.zip](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip).

In [1]:
%matplotlib inline

In [2]:
from papierstat.datasets import load_movielens_dataset
data = load_movielens_dataset()
list(sorted(data))

['links', 'movies', 'ratings', 'tags']

In [3]:
data['movies'].tail(n=2)

,movieId,title,genres
9123,164977,The Gay Desperado (1936),Comedy
9124,164979,"Women of '69, Unboxed",Documentary


In [4]:
rate = data["ratings"]
rate.tail(n=2)

,userId,movieId,rating,timestamp
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [5]:
len(set(rate['userId'])), len(set(rate['movieId']))

(671, 9066)

671 utilisateurs et 9066 films. C'est petit mais assez pour voir la factorisation et le temps que cela prend.